In [ ]:
import os
import statistics
import scipy as sp
import math
import pandas as pd
#pd.get_option("display.max_rows")
#pd.reset_option("display.max_rows")
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as tick
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from statsmodels.regression.linear_model import OLS
from statsmodels.stats.outliers_influence import OLSInfluence
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, RegressorMixin
%matplotlib inline

In [ ]:
os.chdir(' ') #set proper directory depending where you put that project
#data - data for 2019/2020
#data05 - data for 2017/2018
#data1 - data for 2018/19
#data15 - data for 2020/21
data    = pd.read_csv('danelic2019.csv',sep=';',engine='python')
data05  = pd.read_csv('danelic2017.csv',sep=';',engine='python')
data1   = pd.read_csv('danelic2018.csv',sep=';',engine='python')
#data15  = pd.read_csv('danelic2020.csv',sep=';',engine='python')
data1   = pd.DataFrame.append(data,data1)
data1   = pd.DataFrame.append(data1,data05,ignore_index=True)
#data1   = pd.DataFrame.append(data1,data15,ignore_index=True)
data1.sort_values('player')

In [ ]:
#adding dummy variables to dataset
data1 = pd.get_dummies(data1, columns=['league'])
data1 = data1.rename({"league_Bundesliga":"isBundesliga",
                                "league_La Liga":"isLaLiga",
                                "league_Premier League":"isPremierLeague",
                                "league_Ligue 1":"isLigue1",
                                "league_Serie A":"isSerieA"},axis='columns')
data1 = pd.get_dummies(data1,columns=['Season'])
data1 = pd.get_dummies(data1,columns=['foot'])
#deleting potential outliers that actually contribute nothing
data1 = data1[data1['value']>1000000]
data1 = data1[data1['games']>5]
data1 = data1[data1['age']>0]
data1 = data1[data1['height']>0]
data1

In [ ]:
#GOALKEEPERS
dataGK = data1[data1['position2']=='Goalkeeper']
dataGK=dataGK[dataGK['player']!='Emiliano Viviano'] #guy was causing some problems
dataGK

In [ ]:
#was originally used for correlations calculations (visible in appendix)

#dataGK['age']=dataGK['age']*dataGK['age']
#dataGK_cor = dataGK.corr()
#show=pd.Series(dataGK_cor['value'])
#pd.set_option('display.max_rows', None)
#show.sort_values(ascending=False)

In [ ]:
#for my liking
def ln(x):
    return np.log(x)

#dataGK['test']=dataGK['W']/dataGK['MP']
#Creating a linear regression
trainGK, testGK = train_test_split(dataGK, train_size=0.8)
modelGK=smf.ols('ln(value)~age+CL+wins_gk+draws_gk+passes_pct_launched_gk+psnpxg_per_shot_on_target_against+'
                  '+isPremierLeague+isLigue1+clean_sheetsm',data=dataGK)
modelGK1=smf.ols('value~wins_gk+clean_sheets+Pts+W+GDiff+clean_sheets_pct+CL+xGDiff+GF+xG+passes_ground+passes_completed_medium+passes_medium+games+games_starts+minutes_90s+minutes+games_gk+games_starts_gk+minutes_90s_gk+minutes_gk+passes_throws_gk+passes_other_body+passes_completed+passes_received+passes_live+pass_targets+carries+touches_live_ball+passes_pct_long+touches_def_pen_area+passes_completed_short+passes_gk+passes_pressure+passes_pct+def_actions_outside_pen_area_gk+passes_total_distance+psxg_net_gk+touches_def_3rd+passes_short+passes+touches+ball_recoveries+through_balls+dribble_tackles_pct+psxg_net_per90_gk+passes_pct_launched_gk+save_pct+passes_low+xa_net+passes_progressive_distance+WinCL+carry_distance+gca_passes_dead+errors+passes_switches+passes_completed_long+crosses_gk+passes_intercepted+crosses_stopped_gk+dribbles_completed_pct+passes_left_foot+carry_progressive_distance+isPremierLeague+MP+avg_distance_def_actions_gk+saves+draws_gk+assists+goal_kicks+gca+foot_left+isLaLiga+passes_right_foot+shots_on_target_against+passes_pct_short+aerials_won_pct+passes_dead+assists_per90+gca_per90+passes_completed_launched_gk+passes_long+sca_passes_dead+def_actions_outside_pen_area_per90_gk+passes_pct_medium+crosses_stopped_pct_gk+passes_oob+own_goals_against_gk+gca_passes_live+pens_conceded+shots_on_target_pct+throw_ins+psxg_gk+pens_missed_gk+goals_assists_pens_per90+passes_received_pct+height+pens_allowed+goals_assists_per90+passes_launched_gk+npxg_net+pens_att_gk+cards_red+sca+xg_net+sca_passes_live+passes_high+fouled+free_kick_goals_against_gk+cards_yellow+corner_kicks_in+xa+passes_offsides+pens_saved+dribbles_completed+dribble_tackles+assisted_shots+players_dribbled_past+npxg_per_shot+xa_per90+passes_into_penalty_area+pressure_regain_pct+tackles_def_3rd+passes_free_kicks+miscontrols+dribbles+dribbles_vs+passes_head+isSerieA+clearances+corner_kick_goals_against_gk+dribbled_past+corner_kicks+shots_on_target_per90+tackles+goals_against_gk+pressures_def_3rd+tackles_won+dispossessed+tackles_mid_3rd+fouls+shots_total_per90+progressive_passes+offsides+npxg_xa_per90+xg_xa_per90+goals_pens_per90+passes_blocked+touches_mid_3rd+aerials_won+shots_on_target+sca_dribbles+gca_shots+pens_att+pens_made+pens_won+nutmegs+goals_per90+crosses+pressures+blocked_shots+pressure_regains+interceptions+goals_per_shot+shots_total+pressures_mid_3rd+shots_free_kicks+touches_att_pen_area+goals+sca_fouled+pressures_att_3rd+aerials_lost+touches_att_3rd+tackles_att_3rd+xg+goals_per_shot_on_target+own_goals+npxg+sca_shots+npxg_per90+xg_per90+blocks+blocked_passes+sca_per90+crosses_into_penalty_area+passes_into_final_third+D+psnpxg_per_shot_on_target_against+goal_kick_length_avg+foot_right+isBundesliga+isLigue1+passes_length_avg_gk+pct_goal_kicks_launched+losses_gk+pct_passes_launched_gk+age+goals_against_per90_gk+xGA+GA+L+LgRk+gca_dribbles+gca_fouled+gca_og_for+corner_kicks_out+corner_kicks_straight+foot_both+cards_yellow_red+blocked_shots_saves',data=dataGK)    

resultsGK=modelGK.fit()
resultsGK1=modelGK1.fit()
resultsGK_params=resultsGK.params
resultsGK1_params=resultsGK1.params
#Creating a robust regression
modelGKrobust=sm.RLM(modelGK.endog,modelGK.exog,data=dataGK).fit()
#r2_wls = sm.WLS(modelGK.endog, modelGK.exog, weights=modelGKrobust.weights).fit().rsquared
#modelGKrobustridge = sm.WLS(modelGK.endog, modelGK.exog, weights=modelGKrobust.weights).fit_regularized(alpha=0.2, L1_wt=0,start_params=resultsGK_params)
#print(modelGKrobust.summary(),
      #"R^2:", r2_wls)

finalGK1 = sm.regression.linear_model.OLSResults(modelGK, 
                                              modelGKrobust.params, 
                                              modelGK.normalized_cov_params)
finalGK1.summary()
#print(finalGK1.summary())
#print('do not bother about it printing summary like for OLS - this is a robust regression as you can see from the code')

In [ ]:
#VIF 
vif = pd.DataFrame()
vif["features"] = modelGK.exog_names
vif["VIF Factor"] = [variance_inflation_factor(finalGK1.model.exog, i) for i in range(finalGK1.model.exog.shape[1])]
vif

In [ ]:
#testing
#Breusch-Pagan
bptestGK=sm.stats.diagnostic.het_breuschpagan(finalGK1.resid, finalGK1.model.exog)
bptestGK

In [ ]:
#Chowtest
trainGK1=dataGK[dataGK['Season_201819#']==0]
trainGK2=dataGK[dataGK['Season_201819#']==1]
JGK = len(resultsGK.params)-1 #number of coefficients
N1GK = trainGK1.shape[0]
N2GK = trainGK2.shape[0]
RSSdGK=resultsGK.ssr
resultsGKridge1=smf.ols('ln(value)~age+CL+wins_gk+draws_gk+passes_pct_launched_gk+psnpxg_per_shot_on_target_against+'
                  '+isPremierLeague+isLigue1+clean_sheetsm',data=trainGK1).fit()
RSSbGK=resultsGKridge1.ssr
kGK = len(resultsGKridge1.params)-1
resultsGKridge2=smf.ols('ln(value)~age+CL+wins_gk+draws_gk+passes_pct_launched_gk+psnpxg_per_shot_on_target_against+'
                  '+isPremierLeague+isLigue1+clean_sheetsm',data=trainGK2).fit()
RSSnbGK = resultsGKridge2.ssr
ChowGK=((RSSdGK-(RSSbGK+RSSnbGK))/JGK)/((RSSbGK+RSSnbGK)/(N1GK+N2GK-2*kGK))
pGK=sp.stats.f.cdf(ChowGK, JGK, N1GK+N2GK-2*kGK)
print(ChowGK,pGK,JGK)

In [ ]:
predictionsGK=finalGK1.predict(dataGK)
dataGK['predsOLS']=np.exp(predictionsGK)

In [ ]:
#cross validation
class SMWrapper(BaseEstimator, RegressorMixin):
    """ A universal sklearn-style wrapper for statsmodels regressors """
    def __init__(self, model_class, fit_intercept=True):
        self.model_class = model_class
        self.fit_intercept = fit_intercept
    def fit(self, X, y):
        if self.fit_intercept:
            X = sm.add_constant(X)
        self.model_ = self.model_class(y, X)
        self.results_ = self.model_.fit()
        #self.results_ = self.model_.fit_regularized(L1_wt=1, alpha=0.1,start_params=resultsGK_params)
    def predict(self, X):
        if self.fit_intercept:
            X = sm.add_constant(X)
        return self.results_.predict(X)
    
linearcval=cross_val_score(SMWrapper(sm.RLM), modelGK.exog, modelGK.endog, scoring='neg_root_mean_squared_error')
pd.DataFrame(pd.Series(np.transpose(linearcval)))
#robustcval=cross_val_score(SMWrapper(sm.RLM), modelGK.exog, modelGK.endog, scoring='neg_mean_absolute_error')
#pd.DataFrame(pd.concat([pd.Series(np.transpose(linearcval)),pd.Series(np.transpose(robustcval))],axis=1))

In [ ]:
data=dataGK[dataGK['Season_201920#']==1]
#data1=data.sort_values('clean_sheetsm',ascending=False)[0:10]['value']
#data2=data.sort_values('psnpxg_per_shot_on_target_against',ascending=True)[0:10]['value']
#data3=data.sort_values('passes_pct_launched_gk',ascending=False)[0:10]['value']
data=data[['player','value','clean_sheetsm','psnpxg_per_shot_on_target_against','passes_pct_launched_gk']]
data1=data.sort_values('value',ascending=False)[0:10]
pd.DataFrame(data.mean()).transpose()

In [ ]:
def millions(x, pos):
    'The two args are the value and tick position'
    return '%1.1fM' % (x * 1e-6)
formatter = mpl.ticker.FuncFormatter(millions)

dataGK=dataGK[dataGK['clean_sheetsm']>0]
dataGK=dataGK[dataGK['passes_pct_launched_gk']>0]
dataGK=dataGK[dataGK['psnpxg_per_shot_on_target_against']>0]
corrcoef1=np.corrcoef(dataGK['value'],dataGK['clean_sheetsm'])
corrcoef2=np.corrcoef(dataGK['value'],dataGK['passes_pct_launched_gk'])
corrcoef3=np.corrcoef(dataGK['value'],dataGK['psnpxg_per_shot_on_target_against'])
corrcoef1=corrcoef1[0,1]
corrcoef2=corrcoef2[0,1]
corrcoef3=corrcoef3[0,1]

fig, ax = plt.subplots(3, 1, figsize=(12, 12))
sns.regplot(ax=ax[0],x=dataGK['clean_sheetsm'],y=dataGK['value'],data=dataGK,color='g')
sns.regplot(ax=ax[1],x=dataGK['passes_pct_launched_gk'],y=dataGK['value'],data=dataGK,color='blue')
sns.regplot(ax=ax[2],x=dataGK['psnpxg_per_shot_on_target_against'],y=dataGK['value'],data=dataGK,color='orange')

ax[0].yaxis.set_major_formatter(formatter)
ax[0].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[0].annotate("{:.2f}".format(corrcoef1),xy=(0.825,0.85), xycoords="axes fraction")

ax[1].yaxis.set_major_formatter(formatter)
ax[1].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[1].annotate("{:.2f}".format(corrcoef2),xy=(0.825,0.85), xycoords="axes fraction")

ax[2].yaxis.set_major_formatter(formatter)
ax[2].annotate("r=",xy=(0.8,0.85), xycoords="axes fraction")
ax[2].annotate("{:.2f}".format(corrcoef3),xy=(0.825,0.85), xycoords="axes fraction")